In [1]:
import os
import torch
import pandas as pd
import transformers
from datasets import load_dataset
from tqdm import tqdm
import pickle
from pathlib import Path

In [2]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

MODEL_PATH = '../model/sheared_llama_1.3B/'
model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_PATH).to(DEVICE)
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_PATH)
model.to(DEVICE)

cuda:0


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )


In [3]:
# this function extracts the activation vectors for every layer during the forward pass of a prompt and saves them as .pkl files
def process_dataset(dataset_name):
    df = []
    print(f"Saving activations for {dataset_name} dataset!")
    if "GoEmo" in dataset_name:
        df = pd.read_pickle('../datasets/pkl/go_emotions.pkl')
    elif "yelp" in dataset_name:
        df = pd.read_pickle('../datasets/pkl/yelp.pkl')
        
    actis, actis_train, actis_test = [],[],[]
    i = 0
    j = 0
    for index, row in tqdm(df.iterrows(), desc=f"Iterating through all samples from the {DATASET} dataset and extracting the activations", total=df.shape[0]):

        # removing newlines from samples.
        sentence = []
        if "GoEmo" in dataset_name:
            sentence = row['text'].replace('\n', '')
        else:
            sentence = row['sample'].replace('\n', '')
        input_tokens = tokenizer(sentence, return_tensors="pt").to(DEVICE)

        # skip samples with more than 300 tokens, otherwise GPU runs out of memory
        if len(input_tokens.input_ids) > 300: 
            continue
        gen_text = model.forward(input_tokens.input_ids, output_hidden_states=True, return_dict=True)
        hidden_states = []

        #iterating over all layers and storing activations of the last token
        for layer in gen_text['hidden_states']:
            hidden_states.append(layer[0][-1].detach().cpu().numpy())

        if DATASET == "GoEmo":
            actis.append([index,row,hidden_states])
        else:
            # yelp store the labels in column 'sentiment', go emotion stores labels in 'labels' column.
            actis.append([index, sentence, hidden_states, row['sentiment']])

            i += 1
            # save activations in batches
            if i == 10000:
                i = 0
                with open(f'{PATH_TO_ACTIVATION_STORAGE}/{dataset_name}_activations_{j}.pkl', 'wb') as f:
                    pickle.dump(actis, f)
                del actis
                del hidden_states
                actis = []
                j += 1
    
    if DATASET=="GoEmo":
        actis_train = actis[0:4343] # training set
        actis_test = actis[4343:4343+554] # test set
        # we ignore the val set
        with open(f'{PATH_TO_ACTIVATION_STORAGE}/{dataset_name}_activations_train.pkl', 'wb') as f:
                pickle.dump(actis_train, f)
        with open(f'{PATH_TO_ACTIVATION_STORAGE}/{dataset_name}_activations_test.pkl', 'wb') as f:
                pickle.dump(actis_test, f)
    else:    
        # in case the number of samples is not dividable by 10000, we safe the rest
        with open(f'{PATH_TO_ACTIVATION_STORAGE}/{dataset_name}_activations_{j}.pkl', 'wb') as f:
                pickle.dump(actis, f)
    
    del actis
    del hidden_states
    


In [4]:
# DATASET = "yelp", "GoEmo"
DATASET = "GoEmo"

PATH_TO_ACTIVATION_STORAGE=""
if DATASET=="yelp":     PATH_TO_ACTIVATION_STORAGE = "../store_activation/yelp"
elif DATASET=="GoEmo":  PATH_TO_ACTIVATION_STORAGE = "../store_activation/GoEmo"

process_dataset(DATASET)

Saving activations for GoEmo dataset!


Iterating through all samples from the GoEmo dataset and extracting the activations: 100%|██████████| 5410/5410 [02:45<00:00, 32.64it/s]


In [5]:
# DATASET = "yelp", "GoEmo"
DATASET = "GoEmo"

PATH_TO_ACTIVATION_STORAGE=""
if DATASET=="yelp":     PATH_TO_ACTIVATION_STORAGE = "../store_activation/yelp"
elif DATASET=="GoEmo":  PATH_TO_ACTIVATION_STORAGE = "../store_activation/GoEmo"

process_dataset(DATASET)

Saving activations for GoEmo dataset!


Iterating through all samples from the GoEmo dataset and extracting the activations: 100%|██████████| 5410/5410 [02:43<00:00, 33.00it/s]
